In [ ]:
# TensorFlow and tf.keras
import math
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras


#tf version should be 2.5 or higher
tf.__version__

'2.6.0'

In [ ]:
train=pd.read_csv('sign_mnist_train.csv')
test=pd.read_csv('sign_mnist_test.csv')

In [ ]:
train_images=train.values[0:,1:]

test_images=test.values[0:,1:]

In [ ]:
labels = train.values[0:,0]




labels_test = test.values[0:,0]


[ 3  6  2 ... 18 17 23]


In [ ]:
train_images=train_images/255
train_images=train_images.reshape((27455,28,28,1))

test_images=test_images/255
print(test_images)
test_images=test_images.reshape((7172, 28,28,1))
print(test_images)

In [103]:
#setup model
model = keras.Sequential([
   
    
    keras.layers.Conv2D(kernel_size=3, filters=32, use_bias=False, padding='same', input_shape=(28, 28 , 1)),
    keras.layers.BatchNormalization(center=True, scale=False),
    keras.layers.Activation('relu'),
    
    
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    
    
    
    keras.layers.Conv2D(kernel_size=6, filters=64, use_bias=False, padding='same', strides=2),
    keras.layers.BatchNormalization(center=True, scale=False),
    keras.layers.Activation('relu'),
    
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    
   
    
    keras.layers.Conv2D(kernel_size=6, filters=64, use_bias=False, padding='same', strides=2),
    keras.layers.BatchNormalization(center=True, scale=False),
    keras.layers.Activation('relu'),
    
        
    
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    
   
    
    keras.layers.Flatten(),
    
    #keras.layers.Dense(256, use_bias=False),
    #keras.layers.BatchNormalization(center=True, scale=False),
    #keras.layers.Activation('relu'),
    #keras.layers.Dropout(0.3),
    
    #keras.layers.Dense(128, use_bias=False),
    #keras.layers.BatchNormalization(center=True, scale=False),
    #keras.layers.Activation('relu'),
    #keras.layers.Dropout(0.3),
    
    keras.layers.Dense(64, use_bias=False),
    #keras.layers.BatchNormalization(center=True, scale=False),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(25, activation = tf.nn.softmax)
])

In [104]:
#compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

In [105]:
def lr_decay(epoch):
    return 0.01*math.pow(0.6, epoch)

lr_decay_callback=tf.keras.callbacks.LearningRateScheduler(lr_decay)

In [106]:
#train model
model.fit(train_images, labels, validation_split=0.3, epochs=10, batch_size=64, callbacks=[lr_decay_callback])

Epoch 1/10
301/301 [==============================] - 10s 32ms/step - loss: 1.3628 - accuracy: 0.5461 - val_loss: 1.4315 - val_accuracy: 0.5458
Epoch 2/10
301/301 [==============================] - 10s 33ms/step - loss: 0.2661 - accuracy: 0.9050 - val_loss: 0.9701 - val_accuracy: 0.7351
Epoch 3/10
301/301 [==============================] - 10s 33ms/step - loss: 0.1083 - accuracy: 0.9620 - val_loss: 0.0330 - val_accuracy: 0.9928
Epoch 4/10
301/301 [==============================] - 10s 32ms/step - loss: 0.0784 - accuracy: 0.9721 - val_loss: 0.0018 - val_accuracy: 0.9996
Epoch 5/10
301/301 [==============================] - 10s 32ms/step - loss: 0.0652 - accuracy: 0.9769 - val_loss: 8.0423e-04 - val_accuracy: 1.0000
Epoch 6/10
301/301 [==============================] - 10s 32ms/step - loss: 0.0525 - accuracy: 0.9818 - val_loss: 0.0017 - val_accuracy: 0.9998
Epoch 7/10
301/301 [==============================] - 9s 31ms/step - loss: 0.0470 - accuracy: 0.9838 - val_loss: 1.1322e-04 - val_ac

In [107]:
#evaluate
test_loss, test_acc = model.evaluate(test_images,  labels_test)
print('\nTest accuracy:', test_acc)

225/225 [==============================] - 1s 5ms/step - loss: 0.2120 - accuracy: 0.9603

Test accuracy: 0.96026211977005


In [108]:
etalon=model
etalon.save('my_model')

INFO:tensorflow:Assets written to: my_model/assets


INFO:tensorflow:Assets written to: my_model/assets


In [109]:
converter = tf.lite.TFLiteConverter.from_keras_model(etalon)
sign_mnist_model = converter.convert()


INFO:tensorflow:Assets written to: /tmp/tmp9uxzzvpo/assets


INFO:tensorflow:Assets written to: /tmp/tmp9uxzzvpo/assets


In [110]:
f=open('sign_mnist.tflite',  'wb')

In [111]:
f.write(sign_mnist_model)

913092

In [112]:
f.close

<function BufferedWriter.close>

In [113]:
files.download('sign_mnist.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [114]:
import os

print("Size of the 'sign_mnist.tflite' : %.2f Mb"
      %(os.path.getsize('sign_mnist.tflite')/float(2**20)))


Size of the 'sign_mnist.tflite' : 0.87 Mb


In [115]:
import tempfile
import zipfile

test_images=np.float32(test_images)


labels_test=np.float32(labels_test)
interpreter=tf.lite.Interpreter(model_path='sign_mnist.tflite')
interpreter.allocate_tensors()
input_index=interpreter.get_input_details()[0]["index"]
output_index=interpreter.get_output_details()[0]["index"]

def eval_model(x_test, y_test, interpreter):
  total_sceen=0
  num_correct=0

  for img, label in zip(x_test, y_test):
    inp=img.reshape((1, 28, 28, 1))
    total_sceen+=1
    interpreter.set_tensor(input_index, inp)
    interpreter.invoke()
    predictions=interpreter.get_tensor(output_index)
    if np.argmax(predictions)==np.argmax(label):
      num_correct+=1
    if total_sceen % 1000 ==0:
      print('Accuarcy afer %i images: %f' % (total_sceen, float(num_correct)/float(total_sceen)))
  return float(num_correct)/float(total_sceen)

print(eval_model(test_images,  labels_test, interpreter ))

Accuarcy afer 1000 images: 0.043000
Accuarcy afer 2000 images: 0.045000
Accuarcy afer 3000 images: 0.045000
Accuarcy afer 4000 images: 0.043500
Accuarcy afer 5000 images: 0.042400
Accuarcy afer 6000 images: 0.043833
Accuarcy afer 7000 images: 0.045429
0.04573340769659788


In [116]:
cunvert=converter
cunvert.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model=cunvert.convert()

INFO:tensorflow:Assets written to: /tmp/tmp3empxjo4/assets


INFO:tensorflow:Assets written to: /tmp/tmp3empxjo4/assets


In [117]:
g=open('sign_mnist_quant_dyn.tflite', 'wb')
g.write(tflite_quant_model)
g.close
files.download('sign_mnist_quant_dyn.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [118]:

print("Size of the 'sign_mnist_quant_dyn.tflite' : %.2f Mb"
      %(os.path.getsize('sign_mnist_quant_dyn.tflite')/float(2**20)))

Size of the 'sign_mnist_quant_dyn.tflite' : 0.22 Mb


In [119]:
interpreter=tf.lite.Interpreter(model_path='sign_mnist_quant_dyn.tflite')
interpreter.allocate_tensors()
input_index=interpreter.get_input_details()[0]["index"]
output_index=interpreter.get_output_details()[0]["index"]

print(eval_model(test_images,  labels_test, interpreter ))


Accuarcy afer 1000 images: 0.043000
Accuarcy afer 2000 images: 0.045000
Accuarcy afer 3000 images: 0.045333
Accuarcy afer 4000 images: 0.043750
Accuarcy afer 5000 images: 0.042800
Accuarcy afer 6000 images: 0.044167
Accuarcy afer 7000 images: 0.045714
0.046012269938650305


In [120]:

converter = tf.lite.TFLiteConverter.from_keras_model(etalon)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8]
tflite_quant_model = converter.convert()



INFO:tensorflow:Assets written to: /tmp/tmp4g0lxg66/assets


INFO:tensorflow:Assets written to: /tmp/tmp4g0lxg66/assets


In [121]:
g=open('sign_mnist_quant_int16x8.tflite', 'wb')
g.write(tflite_quant_model)
g.close
files.download('sign_mnist_quant_int16x8.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [122]:
print("Size of the 'sign_mnist_quant_int16x8.tflite' : %.2f Mb"
      %(os.path.getsize('sign_mnist_quant_int16x8.tflite')/float(2**20)))

Size of the 'sign_mnist_quant_int16x8.tflite' : 0.22 Mb


In [123]:
interpreter=tf.lite.Interpreter(model_path='sign_mnist_quant_int16x8.tflite')
interpreter.allocate_tensors()
input_index=interpreter.get_input_details()[0]["index"]
output_index=interpreter.get_output_details()[0]["index"]

print(eval_model(test_images,  labels_test, interpreter ))

Accuarcy afer 1000 images: 0.043000
Accuarcy afer 2000 images: 0.045000
Accuarcy afer 3000 images: 0.045333
Accuarcy afer 4000 images: 0.043750
Accuarcy afer 5000 images: 0.042800
Accuarcy afer 6000 images: 0.044167
Accuarcy afer 7000 images: 0.045714
0.046012269938650305


In [124]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model
quantize_annotate_layer=tfmot.quantization.keras.QuantizeConfig

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(etalon)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_4 (QuantizeLa (None, 28, 28, 1)         3         
_________________________________________________________________
quant_conv2d_15 (QuantizeWra (None, 28, 28, 32)        353       
_________________________________________________________________
quant_batch_normalization_22 (None, 28, 28, 32)        97        
_________________________________________________________________
quant_activation_30 (Quantiz (None, 28, 28, 32)        3         
_________________________________________________________________
quant_max_pooling2d_15 (Quan (None, 14, 14, 32)        1         
_________________________________________________________________
quant_conv2d_16 (QuantizeWra (None, 7, 7, 64)          73857     
_________________________________________________________________
quant_batch_normalization_23 (None, 7, 7, 64)         

In [125]:
train_images_subset = train_images[0:1000] 
train_labels_subset = labels[0:1000]

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=500, epochs=1, validation_split=0.1)

/usr/local/lib/python3.7/dist-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


2/2 [==============================] - 2s 417ms/step - loss: 0.5890 - accuracy: 0.7800 - val_loss: 0.2379 - val_accuracy: 1.0000


In [127]:
baseline_model_accuracy = model.evaluate(
    test_images,  labels_test, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
  test_images,  labels_test, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

/usr/local/lib/python3.7/dist-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


Baseline test accuracy: [0.21196623146533966, 0.96026211977005]
Quant test accuracy: 0.9263803958892822


In [136]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model) 
tflite_model = converter.convert()

# Save the model.
with open('sign_mnist_quant_aware_training.tflite', 'wb') as f:
  f.write(tflite_model)
files.download('sign_mnist_quant_aware_training.tflite')  

INFO:tensorflow:Assets written to: /tmp/tmp7zktqi_i/assets


INFO:tensorflow:Assets written to: /tmp/tmp7zktqi_i/assets


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [137]:
print("Size of the 'sign_mnist_quant_aware_training.tflite' : %.2f Mb"
      %(os.path.getsize('sign_mnist_quant_aware_training.tflite')/float(2**20)))

Size of the 'sign_mnist_quant_aware_training.tflite' : 1.73 Mb
